### STEP : Diversity Analysis



#### Example

- [“Moving Pictures” tutorial](https://docs.qiime2.org/2022.8/tutorials/moving-pictures-usage/?highlight=ancom)
- [Biostats ANCOM](http://scikit-bio.org/docs/0.4.2/generated/generated/skbio.stats.composition.ancom.html)


#### Methods
- [composition](https://docs.qiime2.org/2022.8/plugins/available/composition/)
- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.composition.visualizers import ancom
from qiime2.plugins.composition.methods import add_pseudocount

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
metadata_file = '/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [3]:
# Parameters
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
class_col = "above_10"
classifier_file = (
    "/home/lauro/nupeb/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
)
experiment_name = "thayane-PM-paired-end-trim-overlap-04"
manifest_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/manifest-paired.csv"
)
metadata_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/paired-end-metadata.tsv"
)
overlap = 4
phred = 20
replace_files = False
threads = 6
trim = {
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "overlap": 8,
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}
trunc_f = 0
trunc_r = 0


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [6]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T

# ANCOM

Apply Analysis of Composition of Microbiomes (ANCOM) to identify features
that are differentially abundant across groups.

- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

In [7]:
# Select class column
column = metadata_qa.get_column(class_col)

In [8]:
# Create the pseudocount table
composition_tab = add_pseudocount(table = tabs).composition_table

{'pseudocount': 1, 'table': 2210 x 38 <class 'biom.table.Table'> with 6010 nonzero entries (7% dense)}
{'pseudocount': 1, 'table': 2210 x 38 <class 'biom.table.Table'> with 6010 nonzero entries (7% dense)}


In [9]:
# Create visualizations for each parameter combination for the ANCOM  method.
transform_functions = ('sqrt', 'log', 'clr')
difference_functions = ('mean_difference', 'f_statistic')
for t in transform_functions:
    for d in difference_functions:
        print(f"Calculating ANCOM with: {t} {d}")
        try:
            ancom_viz = ancom(
                table = composition_tab, 
                metadata = column, 
                transform_function = t, 
                difference_function = d
            ).visualization
            view_name = os.path.join(qiime_folder, f'ancom-{t}-{d}.qzv')
            ancom_viz.save(view_name)
        except Exception as e:
            print(f"ERROR: Calculating ANCOM with: {t} {d}")
            print(e)

Calculating ANCOM with: sqrt mean_difference
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=38>, 'transform_function': 'sqrt', 'difference_function': 'mean_difference', 'table':       ec39293a8db6cdfc99fecc1decfa59d7  f7a1c2a7dc74eb24446faaa79f519ee6  \
M05                                1.0                              50.0   
M07                                1.0                               1.0   
M102                             231.0                             463.0   
M107                            1449.0                            3653.0   
M112                            1727.0                               1.0   
M120                            8954.0                             706.0   
M121                            3126.0                               1.0   
M125                            7522.0                            4304.0   
M126                            5476.0                            5628.0   
M133                            1990.0      

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: sqrt f_statistic
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=38>, 'transform_function': 'sqrt', 'difference_function': 'f_statistic', 'table':       ec39293a8db6cdfc99fecc1decfa59d7  f7a1c2a7dc74eb24446faaa79f519ee6  \
M05                                1.0                              50.0   
M07                                1.0                               1.0   
M102                             231.0                             463.0   
M107                            1449.0                            3653.0   
M112                            1727.0                               1.0   
M120                            8954.0                             706.0   
M121                            3126.0                               1.0   
M125                            7522.0                            4304.0   
M126                            5476.0                            5628.0   
M133                            1990.0              

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: log mean_difference
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=38>, 'transform_function': 'log', 'difference_function': 'mean_difference', 'table':       ec39293a8db6cdfc99fecc1decfa59d7  f7a1c2a7dc74eb24446faaa79f519ee6  \
M05                                1.0                              50.0   
M07                                1.0                               1.0   
M102                             231.0                             463.0   
M107                            1449.0                            3653.0   
M112                            1727.0                               1.0   
M120                            8954.0                             706.0   
M121                            3126.0                               1.0   
M125                            7522.0                            4304.0   
M126                            5476.0                            5628.0   
M133                            1990.0        

...

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: clr mean_difference
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=38>, 'transform_function': 'clr', 'difference_function': 'mean_difference', 'table':       ec39293a8db6cdfc99fecc1decfa59d7  f7a1c2a7dc74eb24446faaa79f519ee6  \
M05                                1.0                              50.0   
M07                                1.0                               1.0   
M102                             231.0                             463.0   
M107                            1449.0                            3653.0   
M112                            1727.0                               1.0   
M120                            8954.0                             706.0   
M121                            3126.0                               1.0   
M125                            7522.0                            4304.0   
M126                            5476.0                            5628.0   
M133                            1990.0        

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: clr f_statistic
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=38>, 'transform_function': 'clr', 'difference_function': 'f_statistic', 'table':       ec39293a8db6cdfc99fecc1decfa59d7  f7a1c2a7dc74eb24446faaa79f519ee6  \
M05                                1.0                              50.0   
M07                                1.0                               1.0   
M102                             231.0                             463.0   
M107                            1449.0                            3653.0   
M112                            1727.0                               1.0   
M120                            8954.0                             706.0   
M121                            3126.0                               1.0   
M125                            7522.0                            4304.0   
M126                            5476.0                            5628.0   
M133                            1990.0                

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
